# Sem Pormpt

In [9]:
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import SystemMessage
import numpy as np
from IPython.display import display, Markdown

In [10]:
client = OpenAI()

In [ ]:
#Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)
    
#Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

split_texts = text_splitter.split_documents(docs)


#Verificando se foi extraido corretamente
print(split_texts[0])

#Embbeding e Vector Store
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(
    documents=split_texts,
    embedding=embedding_model
)

#Retriver
retriever = vector_store.as_retriever()
# Preparar o contexto
pergunta = "Como faço para congifurar a central para o cliente?"
docs_relevantes = retriever.invoke(pergunta)
context_list = []
for doc in docs_relevantes:
    context_list.append(doc.page_content)

context = "\n\n".join(context_list)
#Instanciando o modelo de IA e enviando a pergunta
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    temperature=0,
    messages=[
        {
            "role": "user",
            "content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"
        }
    ]
)


#Resposta
print(completion.choices[0].message.content)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (901995009.py, line 13)

# Com Prompt

In [ ]:
# Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Extrair texto dos documentos
texts = []
for doc in docs:
    texts.append(doc.page_content)
    
# Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    sseparators=["\n\n", "\n", ".", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_texts = text_splitter.create_documents(texts)

# Embedding e Vector Store
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(
    documents=split_texts,
    embedding=embedding_model
)

# Retriever
retriever = vector_store.as_retriever()

# Pegar os documentos relevantes baseado na pergunta
pergunta = "Como faço para congifurar a central para o cliente?"
docs_relevantes = retriever.invoke(pergunta)

# Preparar o contexto
context_list = []
for doc in docs_relevantes:
    context_list.append(doc.page_content)

context = "\n\n".join(context_list)

# Instanciando o modelo de IA e enviando a pergunta
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", 
         "content": f"Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado, em formato de passo a passo.\n\n{context}"
         },
        {"role": "user", 
        "content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"
         } 
    ]
)

# Resposta
display(Markdown(completion.choices[0].message.content))


Para configurar a central para o cliente, siga os passos abaixo:

1. **Configuração Inicial da Central:**
   - Acesse o PUTTY.
   - Execute os seguintes comandos:
     ```bash
     sensorgw appConfig
     Default server configuration? (y or n): y
     Client WebApp: "TESTE"
     Client Login:
     Client Password:
     CheckConnection:
     CheckModem:
     Carrier: "operadora"
     
     sync
     sensorgw stop
     ```

2. **Configuração do Hamachi:**
   - Continuando no PUTTY, execute os seguintes comandos para o Hamachi:
     ```
     /etc/init.d/logmein-hamachi stop
     rm -Rf /var/lib/logmein-hamachi/
     /etc/init.d/logmein-hamachi start
     hamachi login
     hamachi set-nick "CLIENTE"
     hamachi attach eduardosleal@mcasistemas.com.br
     hamachi do-join 158-893-675 (Senha: mcaUser)
     ```

3. **Adicionar Verificação de Configuração dos Arquivos .xml:**
   - Execute o procedimento de verificação dos arquivos .xml essenciais à central.

4. **Cadastro da Central no Device Manager:**
   - Acesse o [link do Device Manager](http://dm.sensorweb.com.br/login).
   - Clique em "CENTRAIS" na página principal.
   - Insira as informações de rede da central para IP Dinâmico (D.H.C.P) seguindo as instruções do portal.
   - Faça login com as credenciais:
     - Login: teste
     - Senha: teste
   - Teste se a bateria mantém o hardware em funcionamento sem a fonte, desconectando a fonte da central.
   - Teste o botão ON/OFF da Central:
     - Um clique desliga a central.
     - Mantendo o botão pressionado, force o desligamento.

Ao seguir esses passos, você configurou a central para o cliente e a vinculou ao Device Manager para controle de estoque e suporte remoto.

# Com Hyde

In [ ]:
# Carregar PDF
file = "files/Configuração e teste - Central SensorGW.pdf"
loader = PyMuPDFLoader(file)
docs = loader.load()

# Dividir o texto
text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)
split_texts = text_splitter.split_documents(docs)

# Criar embeddings e armazenar em um vetor
embedding_model = OpenAIEmbeddings()
vector_store = FAISS.from_documents(documents=split_texts, embedding=embedding_model)
retriever = vector_store.as_retriever()

# Definir a pergunta
pergunta = "Como é faço para configurar a central para o cliente?"
hyde_model = ChatOpenAI(
    model_name="gpt-3.5-turbo-0125", 
    temperature=0
)
hyde_prompt = f"""
Responda à seguinte pergunta de forma detalhada e clara. Seja preciso e forneça uma explicação passo a passo:

Pergunta: {pergunta}

Utilize as informações abaixo para criar uma resposta completa:
"""
resposta_hipotetica = hyde_model.invoke([SystemMessage(content=hyde_prompt)])

# Criar embedding da resposta hipotética para busca mais precisa
resposta_hipotetica_embedding = embedding_model.embed_query(resposta_hipotetica.content)

# Criar embedding da pergunta original para busca mais precisa
pergunta_embedding = embedding_model.embed_query(pergunta)

# Garantir que ambos os embeddings sejam numpy arrays
pergunta_embedding = np.array(pergunta_embedding)
resposta_hipotetica_embedding = np.array(resposta_hipotetica_embedding)

# Combinar os embeddings da pergunta original e da resposta hipotética
combined_embedding = (pergunta_embedding + resposta_hipotetica_embedding) / 2

# Buscar documentos com o embedding combinado
retrieved_docs = vector_store.similarity_search_by_vector(combined_embedding)

# Criar o contexto com os documentos recuperados
context_list = []
for doc in retrieved_docs:
    context_list.append(doc.page_content)

# Juntar os contextos recuperados em uma única string
context = "\n\n".join(context_list)

# Gerar a resposta final com base no contexto recuperado
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": f"Use apenas as informações abaixo para responder a pergunta do usuário. Seja detalhado e em formato de passo a passo.\n\n{context}"},
        {"role": "user","content": f"Baseado no seguinte contexto:\n\n{context}\n\nResponda à pergunta: {pergunta}"}
    ]
)

# Exibir a resposta gerada
display(Markdown(completion.choices[0].message.content))


Para configurar a central para o cliente, siga os passos abaixo:

1. **Acessar o PUTTY para configurar a central**:
    - Abra o terminal PUTTY.

2. **Realizar a configuração da central para o envio dos dados ao portal do cliente**:
    - Digite o comando: `sensorgw appConfig`
    - Quando perguntado se é a configuração padrão do servidor, responda com `y`.
    - Preencha as informações solicitadas, como:
        - Nome do Client WebApp: "TESTE"
        - Client Login (preencher quando solicitado)
        - Client Password (preencher quando solicitado)
        - CheckConnection
        - CheckModem
        - Carrier: "operadora"

3. **Sincronizar e parar o serviço**:
    - Digite os comandos:
        - `sync`
        - `sensorgw stop`

4. **Configurar o HAMACHI**:
    - Continue no terminal PUTTY e execute os seguintes comandos:
    - Parar o serviço do HAMACHI: `/etc/init.d/logmein-hamachi stop`
    - Remover configurações prévias do HAMACHI: `rm -Rf /var/lib/logmein-hamachi/`
    - Iniciar o serviço do HAMACHI: `/etc/init.d/logmein-hamachi start`
    
5. **Realizar o LOGIN do HAMACHI**:
    - Execute: `hamachi login`

6. **Definir o nome da central no HAMACHI**:
    - Para definir o nome, utilize o comando: `hamachi set-nick “CLIENTE”`

7. **Vincular a conta do HAMACHI**:
    - Use o comando: `hamachi attach eduardosleal@mcasistemas.com.br`

8. **Conectar na conta do HAMACHI**:
    - Digite: `hamachi do-join 158-893-675`
    - Insira a senha: `mcaUser`

9. **Configurar o IP da central para IP Dinâmico (D.H.C.P)**:
    - No navegador, acesse o IP da central.
    - Faça o login usando as credenciais fornecidas na central.

10. **Testar a bateria da central**:
    - Desconecte a fonte da central e verifique se a bateria mantém o hardware em funcionamento.

11. **Testar botão ON/OFF da central**:
    - Certifique-se de que a central desliga com um clique e, caso necessário, teste o modo forçado de desligamento mantendo o botão pressionado.

Esses passos permitem configurar a central para o cliente, garantindo que esteja conectada e pronta para enviar os dados para o portal do cliente.